# Contrastive Language-Image Pretraining with SogCLR

### **Introduction**

In this tutorial, you will learn how to conduct contrastive language-image pretraining by optimizing the [Global Contrastive Loss](https://arxiv.org/abs/2202.12387) (GCL) on a subset of the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset. Also, you will learn how to evaluate the model on retrieval task using the [MSCOCO](https://cocodataset.org/#home) dataset and zero-shot classification task using the [ImageNet](https://www.image-net.org/challenges/LSVRC/index.php) dataset. The code is based on [iSogCLR's](https://github.com/zhqiu/contrastive-learning-iSogCLR) codebase, which includes the implementation of CLIP, SogCLR and iSogCLR.

### Preparation

First, we:

1. Download the source code and data
2. Install required packages

In [5]:
!pip install gdown

In [1]:
#!git clone -b project https://github.com/hgarg97/EfficientCLIPTraining.git iSogCLR

!export PYTHONPATH="$PYTHONPATH:./iSogCLR/bimodal_exps"
!export HUGGINGFACE_HUB_CACHE='./checkpoints/huggingface'
!mkdir checkpoints

In [2]:
# Creating datasets folder

!mkdir datasets
print(1)
!mkdir -p datasets/imagenet
print(2)

1
2


In [6]:
# Downloading and Unzipping clip.tar.gz file

!gdown 1riKYZDPW2QQLTKX4OWDZK7CpCfM5MLg6    # clip.tar.gz

!tar xf clip_train.tar.gz
print(3)

Downloading...
From: https://drive.google.com/uc?id=1riKYZDPW2QQLTKX4OWDZK7CpCfM5MLg6
To: /scratch/user/hgarg97/dlProjectClip/clip_train.tar.gz
100%|██████████████████████████████████████| 4.06M/4.06M [00:00<00:00, 27.4MB/s]
3


In [7]:
# Downloading and Unzipping cc3m_subset_100k.tar.gz file

!gdown 17lYK5zF0GpSZVXlMcPOHD_nucA2qtdrz    # cc3m_subset_100k.tar.gz

!tar xf cc3m_subset_100k.tar.gz -C datasets
print(4)

Downloading...
From (original): https://drive.google.com/uc?id=17lYK5zF0GpSZVXlMcPOHD_nucA2qtdrz
From (redirected): https://drive.google.com/uc?id=17lYK5zF0GpSZVXlMcPOHD_nucA2qtdrz&confirm=t&uuid=c9038775-d52b-49d8-bcc3-e62873c166f3
To: /scratch/user/hgarg97/dlProjectClip/cc3m_subset_100k.tar.gz
100%|███████████████████████████████████████| 3.07G/3.07G [00:18<00:00, 165MB/s]
4


In [8]:
# Downloading and Unzipping ms_coco_val.tar.gz file

!gdown 1XK6L_jV1ImBzLi4_7tOG7gYCJBjzHWzv    # ms_coco_val.tar.gz

!tar xf mscoco_val.tar.gz -C datasets
print(5)

Downloading...
From (original): https://drive.google.com/uc?id=1XK6L_jV1ImBzLi4_7tOG7gYCJBjzHWzv
From (redirected): https://drive.google.com/uc?id=1XK6L_jV1ImBzLi4_7tOG7gYCJBjzHWzv&confirm=t&uuid=911b6d19-65e8-4ed6-b6d2-e21881b7c0b2
To: /scratch/user/hgarg97/dlProjectClip/mscoco_val.tar.gz
100%|█████████████████████████████████████████| 819M/819M [00:05<00:00, 159MB/s]
5


In [9]:
# Downloading and Unzipping val.tar file

!gdown 1-gT5tg3t39TbZ68jd7YpUrHsXmDO-xCy    # val.tar

!tar xf val.tar -C datasets/imagenet
print(6)

Downloading...
From (original): https://drive.google.com/uc?id=1-gT5tg3t39TbZ68jd7YpUrHsXmDO-xCy
From (redirected): https://drive.google.com/uc?id=1-gT5tg3t39TbZ68jd7YpUrHsXmDO-xCy&confirm=t&uuid=49400df1-2cb8-4222-9ab6-e6f7ba0ebc61
To: /scratch/user/hgarg97/dlProjectClip/val.tar
100%|███████████████████████████████████████| 6.75G/6.75G [00:41<00:00, 161MB/s]
6


In [ ]:
# Installing libraries

!pip install -r ./iSogCLR/requirements.txt    # there may be pip warnings/ errors, should be fine to ignore them

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 158 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 33.5 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 37 kB/s s eta 0:00:01
     |████████████████████████████████| 268 kB 39.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 82 kB/s s eta 0:00:01
     |████████████████████████████████| 133 kB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 536 kB 73.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 25.4 MB/s eta 0:00:01
     |████████████████████████████████| 18.2 MB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████| 12.3 MB 15.5 MB/s eta 0:00:01
     |█████████████████████

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |███████████▋                    | 824.6 MB 153.4 MB/s eta 0:00:10

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |██████████████████▍             | 1305.0 MB 153.6 MB/s eta 0:00:07

...

     |████████████████████████▋       | 1745.4 MB 153.9 MB/s eta 0:00:04

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████▌ | 2157.4 MB 154.6 MB/s eta 0:00:01

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 2267.3 MB 5.6 kB/s 
     |████████████████████████████████| 6.1 MB 84.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 7.7 MB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 341 kB 105.4 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 81.2 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 64 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 62.5 MB 15 kB/s s eta 0:00:01
     |████████████████████████████████| 130 kB 21.0 MB/s eta 0:00:01
     |████████████████████████████████| 63.3 MB 32.9 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 1.1 MB 25.1 MB/s eta 0:00:01
     |████████████████████████████████| 27.8 MB 26.2 MB/s eta 0:00:01
     |████████████████████████████████| 96 

# Training

The following command runs the training script to train a ResNet50 (pretrained on ImageNet) and a DistilBERT (pretrained on BookCorpus and English Wikipedia) on the cc3m dataset using the SogCLR loss for 30 epochs with temperature 0.01.

## isogclr_new_v2 + adamp

### Training

In [2]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_v2_and_adamp \
    --init_model \
    --use_amp \
    --ita_type isogclr_new_v2 \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt adamp

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
Traceback (most recent call last):
  File "/content/./iSogCLR/bimodal_exps/clip.py", line 710, in <module>
    main(args)
  File "/content/./iSogCLR/bimodal_exps/clip.py", line 501, in main
    train_stats = train(model, train_loader, optimizer, tokenizer, epoch, max_epoch, warmup_steps, device, lr_scheduler, 
  File "/content/./iSogCLR/bimodal_exps/clip.py", line 66, in train
    for i,(image, text, idx, text_idx) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):
  File "/content/iSogCLR/bimodal_exps/utils.py", line 137, in log_every
    for obj in iterable:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 701, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1465, in _next_data
    return self._process_data(data)
  File "/usr/local/lib/python3

### Evaluation

The following command runs the evaluation script to evaluate the retrieval performance of the trained model on the MSCOCO validation dataset and the zero-shot classification performance on the ImageNet validation dataset. The evaluation command is obtained by appending `--evaluate --checkpoint /path/to/your/checkpoint --zs_dataset imagenet --zs_datafolder /path/to/imagenet/val` to the training command.

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_v2_and_adamp \
    --init_model \
    --use_amp \
    --ita_type isogclr_new_v2 \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/isogclr_new_v2_and_adamp/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

### Benchmarks

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |